# Week 2: Analyst Agent Testing

This notebook tests the complete Scout → Analyst pipeline with real data.

## Setup

In [ ]:
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / "src"))

from dotenv import load_dotenv
load_dotenv(Path.cwd().parent / "config" / ".env")

import json
from datetime import datetime

from agents.scout import ScoutAgent
from agents.analyst import AnalystAgent
from core.grok_client import GrokClient
from core.alerts import AlertManager
from data.db import Database

print("✅ Imports successful")

## Test 1: Scout Agent

Fetch breakthrough signals from news sources.

In [ ]:
# Initialize Scout
scout = ScoutAgent()

# Fetch signals
scout_result = scout.execute({
    "days_back": 3,
    "max_results": 15,
    "min_relevance": 6
})

print(f"Success: {scout_result['success']}")
print(f"Total fetched: {scout_result['total_fetched']}")
print(f"Total filtered: {scout_result['total_filtered']}")
print(f"\nTop 5 signals:")

for i, article in enumerate(scout_result['articles'][:5], 1):
    print(f"\n{i}. [{article['relevance_score']}/10] {article['title']}")
    print(f"   Source: {article['source']}")
    print(f"   Keywords: {', '.join(article['matched_keywords'][:3])}")

## Test 2: Analyst Agent (Mock Mode)

Test Analyst with mock Grok responses to verify parsing logic.

In [ ]:
from unittest.mock import Mock

# Create mock Grok client
mock_grok = Mock()
mock_grok.model = "grok-beta-mock"
mock_grok.analyze_with_prompt = Mock(return_value="""
IMPACT SCORE: 8

SENTIMENT: bullish

30-DAY OUTLOOK: Expect 10-15% price appreciation as the market digests this breakthrough. Strong institutional interest likely.

KEY INSIGHT: This development solidifies the company's position in the AI infrastructure market and opens new revenue streams.

RISKS:
- Execution risk on scaling production
- Competitive response from AMD/Intel
- Regulatory headwinds on AI chip exports

SCENARIOS:
- 5yr: Market share expands to 75%, stock 2-3x from current levels
- 10yr: AI infrastructure becomes $300B+ market, maintains 60% share
- 20yr: Quantum-AI transition, but ecosystem lock-in persists
""")

# Initialize Analyst with mock
analyst_mock = AnalystAgent(grok_client=mock_grok)

# Analyze top 3 Scout signals
analyst_result = analyst_mock.execute({
    "articles": scout_result['articles'][:3],
    "max_analyses": 3
})

print(f"Success: {analyst_result['success']}")
print(f"Total analyzed: {analyst_result['total_analyzed']}")
print(f"Grok available: {analyst_result['grok_available']}")

for analysis in analyst_result['analyses']:
    print(f"\n{'='*80}")
    print(f"Article: {analysis['article_title']}")
    print(f"Impact Score: {analysis['impact_score']}/10")
    print(f"Sentiment: {analysis['sentiment'].upper()}")
    print(f"30-Day Outlook: {analysis['price_target_30d']}")
    print(f"Key Insight: {analysis['key_insight']}")
    print(f"\nRisks:")
    for risk in analysis['risks']:
        print(f"  - {risk}")
    print(f"\nScenarios:")
    for timeframe, scenario in analysis['scenarios'].items():
        if scenario != "N/A":
            print(f"  - {timeframe}: {scenario}")

## Test 3: Analyst Agent (Real Grok API)

Test with real Grok API if configured.

**Note:** This will make actual API calls. Skip if XAI_API_KEY is not set.

In [ ]:
import os

if os.getenv("XAI_API_KEY"):
    print("✅ XAI_API_KEY found, testing real Grok API...\n")
    
    # Initialize real Analyst
    try:
        grok = GrokClient()
        analyst = AnalystAgent(grok_client=grok)
        
        # Analyze top signal only (to save API credits)
        analyst_result_real = analyst.execute({
            "articles": scout_result['articles'][:1],
            "max_analyses": 1
        })
        
        if analyst_result_real['success']:
            analysis = analyst_result_real['analyses'][0]
            
            print(f"Article: {analysis['article_title']}")
            print(f"Impact Score: {analysis['impact_score']}/10")
            print(f"Sentiment: {analysis['sentiment'].upper()}")
            print(f"\nKey Insight:\n{analysis['key_insight']}")
            print(f"\n30-Day Outlook:\n{analysis['price_target_30d']}")
            print(f"\nRisks:")
            for risk in analysis['risks']:
                print(f"  - {risk}")
            print(f"\nLong-Term Scenarios:")
            for timeframe, scenario in analysis['scenarios'].items():
                if scenario != "N/A":
                    print(f"  - {timeframe}: {scenario}")
            
            print(f"\n✅ Real Grok API test successful!")
            print(f"Model: {analysis['grok_model']}")
        else:
            print("❌ Analyst execution failed")
    
    except Exception as e:
        print(f"❌ Error: {e}")
else:
    print("⚠️ XAI_API_KEY not set, skipping real API test")
    print("Add XAI_API_KEY to config/.env to test real Grok integration")

## Test 4: High-Impact Filtering

Test filtering high-impact signals.

In [ ]:
# Use mock analyst results
high_impact = analyst_mock.get_high_impact_signals(
    analyst_result['analyses'],
    threshold=8
)

print(f"High-impact signals (≥8/10): {len(high_impact)}")

for signal in high_impact:
    print(f"\n⚡ {signal['impact_score']}/10 - {signal['article_title']}")
    print(f"   Sentiment: {signal['sentiment'].upper()}")
    print(f"   Insight: {signal['key_insight'][:100]}...")

## Test 5: Alert System

Test creating alerts for high-impact signals.

In [ ]:
# Initialize AlertManager
alert_manager = AlertManager()

# Create alerts for high-impact signals
alert_ids = []
for analysis in analyst_result['analyses']:
    alert_id = alert_manager.create_high_impact_alert(analysis, threshold=7)
    if alert_id:
        alert_ids.append(alert_id)
        print(f"✅ Created alert ID {alert_id} for: {analysis['article_title']}")

print(f"\nTotal alerts created: {len(alert_ids)}")

## Test 6: Retrieve Recent Alerts

Test retrieving and displaying recent alerts.

In [ ]:
# Get recent alerts
recent_alerts = alert_manager.get_recent_alerts(
    hours=24,
    min_severity="MEDIUM",
    unacknowledged_only=False
)

print(f"Recent alerts (last 24h): {len(recent_alerts)}\n")

for alert in recent_alerts[:5]:
    print(f"[{alert['severity']}] {alert['title']}")
    print(f"Impact: {alert['impact_score']}/10")
    print(f"Created: {alert['created_at']}")
    print(f"Acknowledged: {'Yes' if alert['acknowledged'] else 'No'}")
    print()

# Get alert summary
summary = alert_manager.get_alert_summary(hours=24)
print(f"Alert Summary (24h):")
print(f"  Total: {summary['total']}")
print(f"  Unacknowledged: {summary['unacknowledged']}")
print(f"  By Severity: {summary['by_severity']}")

## Test 7: Fallback Analysis

Test fallback behavior when Grok API is unavailable.

In [ ]:
# Create Analyst without Grok
analyst_no_grok = AnalystAgent(grok_client=None)

# Try to analyze
fallback_result = analyst_no_grok.execute({
    "articles": scout_result['articles'][:2],
    "max_analyses": 2
})

print(f"Success: {fallback_result['success']}")
print(f"Grok available: {fallback_result['grok_available']}")
print(f"\nFallback analyses:")

for analysis in fallback_result['analyses']:
    print(f"\n{analysis['article_title']}")
    print(f"  Impact: {analysis['impact_score']}/10 (based on relevance)")
    print(f"  Sentiment: {analysis['sentiment']}")
    print(f"  Is Fallback: {analysis.get('is_fallback', False)}")
    print(f"  Insight: {analysis['key_insight']}")

## Test 8: Database Persistence

Verify signals and alerts are persisted correctly.

In [ ]:
# Initialize database
db = Database()

# Cache Scout signals
for article in scout_result['articles'][:5]:
    db.cache_scout_signal(article)

print("✅ Cached Scout signals to database")

# Retrieve cached signals
cached = db.get_cached_signals(limit=5)
print(f"\nCached signals in DB: {len(cached)}")

for signal in cached:
    print(f"  [{signal['relevance_score']}/10] {signal['title']}")

print("\n✅ Database persistence test complete")

## Summary

All tests completed! Review the results above to verify:

1. ✅ Scout Agent fetches and filters news
2. ✅ Analyst Agent parses Grok responses correctly
3. ✅ Real Grok API integration works (if configured)
4. ✅ High-impact filtering works
5. ✅ Alert system creates and retrieves alerts
6. ✅ Fallback analysis works when Grok unavailable
7. ✅ Database persistence works

**Next Steps:**
- Run the Streamlit dashboard: `streamlit run src/app.py`
- Check Daily Brief page for Scout→Analyst pipeline
- Verify high-impact alerts on Overview page